In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

Loading BokehJS ...

In [2]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

In [3]:
def init_graph_attr(G, AdjFile, df, columns):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    capability_vector = list(zip(*(df[col] for col in columns)))
    node_attri_dict = dict(zip(df["District"], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["nodeStability"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

In [4]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1

def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

In [5]:
def get_node_stress(G,dim):
    stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return stress_dict

In [6]:
def get_node_stability(G,dim):
    stability_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStability"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stability_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStability"]
        except(KeyError):
            pass
    return stability_dict

In [7]:
G = nx.Graph()
df = pd.read_csv('KAG 2016-17/Agriculture/Agriculture_KAG_2016_17.csv')
adjacency_file = 'Karnataka_District_Adjacency_File.xlsx'

# existing_data = pd.DataFrame(df['District'])

existing_data = pd.read_csv('result.csv')
# adjacency_file = "KAG 2016-17/Karnataka_District_Adjacency_File.xlsx"

In [8]:
def calculate_and_return_initial_stability(df, columns, G, adjacency_file = adjacency_file):
    dim = len(columns)
    col_to_pass = []

    for column in columns:
        column_values = df[column].values.reshape(-1, 1)
        scaler = MinMaxScaler()
        normalized_column_values = scaler.fit_transform(column_values)
        df[f"Normalized_{column}"] = normalized_column_values
        col_to_pass.append(f"Normalized_{column}")

    init_graph_attr(G, adjacency_file, df, col_to_pass)
    initial_stability = get_node_stability(G, dim)
    df["Initial Stability"] = df["District"].map(initial_stability)

    return df["Initial Stability"]

In [9]:
columns_to_normalize = ["Maize_Production"]
dim = 1
for col in columns_to_normalize:
    column_values = df[col].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    normalized_values = scaler.fit_transform(column_values)
    df[f"Normalized_{col}"] = normalized_values.flatten()
    init_graph_attr(G,adjacency_file , df,[f"Normalized_{col}"] )
    initial_stability = get_node_stability(G, dim)
    df[f"Initial Stability_{col}"] = df["District"].map(initial_stability)

In [10]:
df

,District,Nitrogen,Phosphorus,Potassium,TotalNPK,Rice_Production,Jowar_Production,Bajra_Production,Maize_Production,Ragi_Production,...,SowingSeedsDistributed_Bajra,SowingSeedsDistributed_Safflower,SowingSeedsDistributed_Groundnut,SowingSeedsDistributed_Sunflower,SowingSeedsDistributed_Soyabean,RRB_AgricultureLoan,DCCBank_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,Normalized_Maize_Production,Initial Stability_Maize_Production
0,BENGALURU,13519,7415,2376,23310,3376,851,0,2879,62023,...,0.00,0.00,8.25,0.00,0.00,3450.06,923.83,840.78,0.004851,0.974079
1,BENGALURU(R),9819,7757,1683,19259,2354,0,0,42583,80284,...,0.00,0.00,10.65,0.00,0.00,10539.94,10040.50,1172.91,0.071744,0.991418
2,RAMANAGARA,5090,1593,789,7472,12128,0,3,10450,136422,...,0.00,0.00,217.80,0.00,0.00,9921.94,20868.05,413.86,0.017606,0.923391
3,CHITRADURGA,20375,11971,3538,35884,4566,13301,1085,271975,93320,...,66.00,0.00,32035.00,348.00,0.00,105785.02,17273.65,853.14,0.458222,0.933432
4,DAVANAGERE,54418,33858,11368,99644,489505,23072,794,593544,18774,...,11.62,0.00,49693.50,623.10,0.00,80725.12,24202.58,1081.95,1.000000,0.398096
5,KOLAR,11237,7842,3111,22190,864,0,0,1759,66420,...,0.00,0.00,3015.50,0.00,0.00,70194.91,5775.47,1715.02,0.002964,0.918189
6,CHIKKABALLAPURA,14398,10512,3447,28357,2698,0,0,105491,57499,...,0.00,0.00,6536.90,0.00,0.00,48782.34,8939.40,1827.19,0.177731,0.884725
7,SHIVAMOGGA,27793,17431,11385,56609,384974,756,2,226532,1067,...,0.00,0.00,30.00,0.00,0.00,69036.55,31135.21,1844.32,0.381660,0.986495
8,TUMAKURU,21285,12325,4538,38148,26042,2652,290,66868,334611,...,2.10,0.00,31094.20,30.00,0.00,104566.89,41077.53,1802.15,0.112659,0.909238
9,CHIKKAMAGALURU,29822,19107,18933,67863,110533,9080,0,61353,52776,...,0.00,0.00,1485.30,79.54,0.00,32999.48,45035.20,1322.43,0.103367,0.744033


In [11]:
# df["Initial Stability_Rice_Production_223"]

In [12]:
existing_data = pd.DataFrame(existing_data['District'])

In [13]:
# def calculate_impact_score(df, base_column, capability_vector, intervention):
#     # Perform simple linear regression
#     X = sm.add_constant(df[base_column])
#     y = df[capability_vector]
#     model = sm.OLS(y, X).fit()

#     # Get coefficients and intercept
#     m, c = model.params[base_column], model.params['const']
#     print(c)
#     # Predicted change in capability_vector for base_column + change_percentage% and base_column - change_percentage%
#     base_column_increase = (1 + intervention/100) * df[base_column]
#     base_column_decrease = (1 - intervention/100) * df[base_column]

#     rp_new_plus = m * base_column_increase + c 
#     rp_new_minus = m * base_column_decrease + c 

#     vector_plus = rp_new_plus - m * df[base_column] - c  
#     vector_minus = rp_new_minus - m * df[base_column] - c

#     # Generate normalized values using MinMaxScaler
#     scaler = MinMaxScaler()

#     normalized_vector_plus = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
#     normalized_vector_minus = scaler.fit_transform(vector_minus.values.reshape(-1, 1))

#     # Create new DataFrame with appropriate column names
#     result_df = pd.DataFrame({
#         f'{capability_vector} ({base_column} +{intervention}%)': rp_new_plus,
#         f'{capability_vector} ({base_column} -{intervention}%)': rp_new_minus,
#         f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_vector_plus.flatten(),
#         f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_vector_minus.flatten()
#     })

#     return result_df

In [14]:
# base_Column = "TotalNPK_315"
# CapabilityVector = "Jowar_Yield_278"
# change_percentage = 20

# result_df = calculate_impact_score(df, base_Column, CapabilityVector, change_percentage)

# result_df.head()

In [15]:
# def merge_columns_into_dataframe(result_df, existing_data):
#     for column in result_df.columns:
#         existing_data[column] = result_df[column]
#     return existing_data

# # Assuming existing_data is already defined
# existing_data = merge_columns_into_dataframe(result_df, existing_data)
# existing_data.head()

In [16]:
# result_df.head()

In [17]:
existing_data.head()

,District
0,BENGALURU
1,BENGALURU(R)
2,RAMANAGARA
3,CHITRADURGA
4,DAVANAGERE


In [18]:
dim = 1

In [19]:
def calculate_and_map_stability(G, existing_data, adjacency_file, columns_to_pass, inter, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    NPK_stability = get_node_stability(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability(NPK {'+' if inter >= 0 else '-'} {abs(inter)}%)"
    stability_column = existing_data["District"].map(NPK_stability)

    return stability_column, stability_column_name

In [20]:
def calculate_and_map_stress(G, existing_data, adjacency_file, columns_to_pass, inter, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    NPK_stress = get_node_stress(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability(NPK {'+' if inter >= 0 else '-'} {abs(inter)}%)"
    stability_column = existing_data["District"].map(NPK_stress)

    return stability_column, stability_column_name

## 1D Capability Vector

In [21]:
from bokeh.models import Span, Label
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

# Define the function to calculate and visualize impact stability
def calc_and_vis_impact_stability_1D(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
    # Calculate Impact Score
    X = sm.add_constant(df[base_column])
    y = df[capability_vector]
    model = sm.OLS(y, X).fit()
    
    m, c = model.params[base_column], model.params['const']

    base_column_increase = (1 + intervention/100) * df[base_column]
    base_column_decrease = (1 - intervention/100) * df[base_column]

    new_vector_increase = m * base_column_increase + c
    new_vector_decrease = m * base_column_decrease + c
    
    vector_plus = new_vector_increase - m * df[base_column] - c  
    vector_minus = new_vector_decrease - m * df[base_column] - c

    scaler = MinMaxScaler()
    normalized_change_increase_vector = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
    normalized_change_decrease_vector = scaler.fit_transform(vector_minus.values.reshape(-1, 1))
    normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))
    normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))

    result_df = pd.DataFrame({
        f'{capability_vector} ({base_column} +{intervention}%)': new_vector_increase,
        f'{capability_vector} ({base_column} -{intervention}%)': new_vector_decrease,
        f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
        f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
        f'Impact Score {capability_vector} ({base_column} +{intervention}%)': normalized_change_increase_vector.flatten(),
        f'Impact Score {capability_vector} ({base_column} -{intervention}%)': normalized_change_decrease_vector.flatten(),
    })
    
    # Merge Columns
    for column in result_df.columns:
        existing_data[column] = result_df[column]

    # Calculate Stability and add to DataFrame
    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite intervention and add to DataFrame
    opposite_intervention = -intervention
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite intervention and add to DataFrame
    opposite_intervention = -intervention
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    # Visualize with Plotly for +intervention
    fig_increase = px.scatter(existing_data,
                              x=stability_column_name,
                              y=f'Impact Score {capability_vector} ({base_column} +{intervention}%)',
                              title=f'Impact vs Stability ({capability_vector} - {base_column} +{intervention}%)',
                              labels={'x': f'Stability ({base_column} +{intervention}%)', 'y': f'Impact ({capability_vector})'},
                              size_max=8,
                              width=800,
                              height=500)
    
    fig_increase.update_traces(hoverlabel=dict(bgcolor='grey', font=dict(color='white')))

    # Calculate average values from the DataFrame for +intervention
    avg_x_increase = existing_data[stability_column_name].mean()
    avg_y_increase = existing_data[f'Normalized {capability_vector} ({base_column} +{intervention}%)'].mean()

    # Add average lines for +intervention
    fig_increase.add_shape(
        type='line',
        x0=avg_x_increase,
        x1=avg_x_increase,
        y0=fig_increase.data[0].y.min(),
        y1=fig_increase.data[0].y.max(),
        line=dict(color='red', width=2)
    )
    fig_increase.add_shape(
        type='line',
        x0=fig_increase.data[0].x.min(),
        x1=fig_increase.data[0].x.max(),
        y0=avg_y_increase,
        y1=avg_y_increase,
        line=dict(color='blue', width=2)
    )

    # Add labels for average lines for +intervention
    fig_increase.add_annotation(
        x=avg_x_increase,
        y=fig_increase.data[0].y.max(),
        text=f'Avg {stability_column_name}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='red',
        ax=0,
        ay=-40,
        font=dict(color='red')
    )
    fig_increase.add_annotation(
        x=fig_increase.data[0].x.max(),
        y=avg_y_increase,
        text=f'Avg Normalized {capability_vector}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='blue',
        ax=-40,
        ay=0,
        font=dict(color='blue')
    )

    # Add tooltips for +intervention
    fig_increase.update_traces(hoverinfo='text+name',
                               hovertext=["District: " + str(d) +
                                          f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} +{intervention}%): {x}"
                                          for x, y, d in zip(fig_increase.data[0].x, fig_increase.data[0].y, existing_data['District'])])

    # Visualize with Plotly for -intervention
    fig_decrease = px.scatter(existing_data,
                              x=stability_column_name_opposite,
                              y=f'Impact Score {capability_vector} ({base_column} -{intervention}%)',
                              title=f'Impact vs Stability ({capability_vector} - {base_column} -{intervention}%)',
                              labels={'x': f'Stability ({base_column} -{intervention}%)', 'y': f'Impact ({capability_vector})'},
                              size_max=8,
                              width=800,
                              height=500)

    # Calculate average values from the DataFrame for -intervention
    avg_x_decrease = existing_data[stability_column_name_opposite].mean()
    avg_y_decrease = existing_data[f'Normalized {capability_vector} ({base_column} -{intervention}%)'].mean()

    # Add average lines for -intervention
    fig_decrease.add_shape(
        type='line',
        x0=avg_x_decrease,
        x1=avg_x_decrease,
        y0=fig_decrease.data[0].y.min(),
        y1=fig_decrease.data[0].y.max(),
        line=dict(color='red', width=2)
    )
    fig_decrease.add_shape(
        type='line',
        x0=fig_decrease.data[0].x.min(),
        x1=fig_decrease.data[0].x.max(),
        y0=avg_y_decrease,
        y1=avg_y_decrease,
        line=dict(color='blue', width=2)
    )

    # Add labels for average lines for -intervention
    fig_decrease.add_annotation(
        x=avg_x_decrease,
        y=fig_decrease.data[0].y.max(),
        text=f'Avg {stability_column_name_opposite}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='red',
        ax=0,
        ay=-40,
        font=dict(color='red')
    )
    fig_decrease.add_annotation(
        x=fig_decrease.data[0].x.max(),
        y=avg_y_decrease,
        text=f'Avg Normalized {capability_vector}',
        showarrow=True,
        arrowhead=4,
        arrowcolor='blue',
        ax=-40,
        ay=0,
        font=dict(color='blue')
    )

    # Add tooltips for -intervention
    fig_decrease.update_traces(hoverinfo='text+name',
                               hovertext=["District: " + str(d) +
                                          f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} -{intervention}%): {x}"
                                          for x, y, d in zip(fig_decrease.data[0].x, fig_decrease.data[0].y, existing_data['District'])])

    # Show the plots
    fig_increase.show()
    fig_decrease.show()

    return existing_data

base_Column = "TotalNPK"
CapabilityVector = "Rice_Production"
change_percentage = 20
inter = [10,20]
dim = 1
# Call the function and get the handles for the two plots
new_data_20 = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data_20.head()


,District,Rice_Production (TotalNPK +20%),Rice_Production (TotalNPK -20%),Normalized Rice_Production (TotalNPK +20%),Normalized Rice_Production (TotalNPK -20%),Impact Score Rice_Production (TotalNPK +20%),Impact Score Rice_Production (TotalNPK -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.098721,0.901279,0.959717,0.959717
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.075418,0.924582,0.973330,0.973330
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.007616,0.992384,0.853066,0.853066
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.171050,0.828950,0.733313,0.733313
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.537816,0.462184,0.827968,0.827968


In [22]:
# ref = new_data[["District","Impact Score Rice_Production_223 (TotalNPK_315 +10%)","Impact Score Rice_Production_223 (TotalNPK_315 -10%)"]]

In [23]:
# cols_to_concat = ['Impact Score Rice_Production_223 (TotalNPK_315 +20%)', 'Impact Score Rice_Production_223 (TotalNPK_315 -20%)']
# res = pd.concat([ref, new_data_20[cols_to_concat]], axis=1)

In [24]:
# res.to_csv('res1.csv',index = False)

In [25]:
change_percentage = 10
dim = 1

# Call the function and get the handles for the two plots
new_data = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data.head()

,District,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Impact Score Rice_Production (TotalNPK +10%),Impact Score Rice_Production (TotalNPK -10%),New Stability(NPK + 10%),New Stability(NPK - 10%)
0,BENGALURU,68479.019211,57507.638667,0.098721,0.098721,0.098721,0.901279,0.959717,0.959717
1,BENGALURU(R),57992.196208,48927.510755,0.075418,0.075418,0.075418,0.924582,0.973330,0.973330
2,RAMANAGARA,27479.191334,23962.324950,0.007616,0.007616,0.007616,0.992384,0.853066,0.853066
3,CHITRADURGA,101029.330849,84139.711825,0.171050,0.171050,0.171050,0.828950,0.733313,0.733313
4,DAVANAGERE,266084.831886,219185.121765,0.537816,0.537816,0.537816,0.462184,0.827968,0.827968


In [26]:

# base_Column = "TotalNPK"
# CapabilityVector = "Maize_Production"
# change_percentage = 20
# dim = 1

# # Call the function and get the handles for the two plots
# new_data = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# # Display the resulting DataFrame
# new_data.head()

In [27]:
# Save the resulting DataFrame to a new CSV file if needed
# existing_data.to_csv('result.csv', index=False)

In [28]:
# from bokeh.models import Span, Label
# from bokeh.plotting import figure, show, output_notebook
# from bokeh.models.sources import ColumnDataSource
# from bokeh.models.tools import HoverTool
# import statsmodels.api as sm
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go

# def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
#     # Calculate min and max values for changes made by each column in capability_vector
#     min_value_increase = float('inf')
#     max_value_increase = float('-inf')
#     min_value_decrease = float('inf')
#     max_value_decrease = float('-inf')
#     col_pass_plus = []
#     col_pass_minus = []
    
#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         change_vector_increase = m * base_column_increase + c
#         change_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = change_vector_increase - m * df[base_column] - c  
#         vector_minus = change_vector_decrease - m * df[base_column] - c

#         # Update min and max values for each column's changes
#         min_value_increase = min(min_value_increase, vector_plus.min())
#         max_value_increase = max(max_value_increase, vector_plus.max())

#         min_value_decrease = min(min_value_decrease, vector_minus.min())
#         max_value_decrease = max(max_value_decrease, vector_minus.max())

#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         new_vector_increase = m * base_column_increase + c
#         new_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = change_vector_increase - m * df[base_column] - c  
#         vector_minus = change_vector_decrease - m * df[base_column] - c

#         # Normalize changes manually using the calculated min and max values
#         scaler3 = MinMaxScaler()
#         normalized_new_increase_vector = scaler3.fit_transform(new_vector_increase.values.reshape(-1, 1))
    
#         scaler4 = MinMaxScaler()
#         normalized_new_decrease_vector = scaler4.fit_transform(new_vector_decrease.values.reshape(-1, 1))
        
#         normalized_change_increase_vector = (vector_plus - min_value_increase) / (max_value_increase - min_value_increase)
#         normalized_change_decrease_vector = (vector_minus - min_value_decrease) / (max_value_decrease - min_value_decrease)

#         result_df = pd.DataFrame({
#             f'{cap_col} ({base_column} +{intervention}%)': change_vector_increase,
#             f'{cap_col} ({base_column} -{intervention}%)': change_vector_decrease,
#             f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
#             f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
#             f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
#             f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,
#         })

#         # Merge Columns
#         for column in result_df.columns:
#             if column in result_df.columns:
#                 existing_data[column] = result_df[column]

#         # Calculate Stability and add to DataFrame
#         columns_to_pass = f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
#         col_pass_plus.append(columns_to_pass)

#         # Calculate Stability for the opposite intervention and add to DataFrame
#         opposite_intervention = -intervention
#         columns_to_pass_opposite = f"Normalized {cap_col} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"
#         col_pass_minus.append(columns_to_pass_opposite)
        
#     stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

#     existing_data[stability_column_name] = stability_column

#     stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

#     existing_data[stability_column_name_opposite] = stability_column_opposite
        

# # Visualize with Plotly for +intervention
#     fig_increase = go.Figure()
#     color_palette = ['red', 'blue', 'green', 'orange', 'purple']  # Add more colors if needed

#     for i, cap_col in enumerate(capability_vector):
#         # Scatter plot for +intervention
#         scatter_increase = go.Scatter(
#             x=existing_data[f"{stability_column_name}"],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=8, color=color_palette[i]),
#             name=f'{cap_col} +{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )

#         fig_increase.add_trace(scatter_increase)

#     # Update layout for +intervention plot
#     fig_increase.update_layout(
#         title=f'Impact vs Stability (+{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Visualize with Plotly for -intervention
#     fig_decrease = go.Figure()

#     for i, cap_col in enumerate(capability_vector):
#         # Scatter plot for -intervention
#         scatter_decrease = go.Scatter(
#             x=existing_data[stability_column_name_opposite],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=8, color=color_palette[i]),
#             name=f'{cap_col} -{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )

#         fig_decrease.add_trace(scatter_decrease)

#     # Update layout for -intervention plot
#     fig_decrease.update_layout(
#         title=f'Impact vs Stability (-{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Show plots
#     fig_increase.show()
#     fig_decrease.show()

#     return existing_data, fig_increase, fig_decrease

# # Assuming df is your DataFrame
# base_column = "TotalNPK"
# capability_vector = ["Rice_Production", "Jowar_Yield"]
# change_percentage = 20
# dim = 1

# # Call the function and get the handles for the two plots
# new_data, handle_increase, handle_decrease = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)

# # Display the resulting DataFrame
# new_data.head()


In [29]:
# new_data.head()

In [30]:
# new_data.to_csv('test.csv', index=False)

## 2D Capability vector

In [31]:
res_df = pd.DataFrame()

In [32]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import cycle

def calc_and_vis_impact_stability_2D1(G, existing_data, adjacency_file, base_column, capability_vector, change_percentages, dim):
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    for intervention in change_percentages:
        for cap_col in capability_vector:
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            m, c = model.params[base_column], model.params['const']

            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())
            
    
    result_df = pd.DataFrame()
    res1_df = pd.DataFrame()
    
    for intervention in change_percentages:
        col_pass_plus = []
        col_pass_minus = []
        for cap_col in capability_vector:
            # Calculate Impact Score
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']
            # print(m,c)
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )*100/(y)
            percent_change_dec = (new_vector_decrease - (y) )*100/(y)

            result_df = pd.concat([result_df,
                pd.DataFrame({
                    f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                    f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                    f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                    f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
                    f'Percent Change in {cap_col} ({base_column} +{intervention}%)': percent_change_inc,
                    f'Percent Change in {cap_col} ({base_column} -{intervention}%)': percent_change_dec,
                })
            ], axis=1)

            columns_to_pass = f'{cap_col} ({base_column} +{intervention}%)'
            col_pass_plus.append(columns_to_pass)
            
            # Calculate Stability for the opposite intervention and add to DataFrame
            opposite_intervention = -intervention
            columns_to_pass_opposite = f'{cap_col} ({base_column} -{intervention}%)'
            col_pass_minus.append(columns_to_pass_opposite)
        
        for column in result_df.columns:
            existing_data[column] = result_df[column]

        # print(col_pass_minus)
        # print(col_pass_plus)
        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

        # normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())

        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
        # existing_data[stability_column_name_modified] = normalized_stability_vector
        
        stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

        # Normalize stability values for the opposite intervention
        # normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
        stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
        # existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite

###################
        stress_column, stress_column_name = calculate_and_map_stress(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)
        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stress_column_name_modified = stress_column_name + "_" + "_".join(capability_first_words)        
        stress_column_opposite, stress_column_name_opposite = calculate_and_map_stress(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)
        stress_column_name_opposite_modified = stress_column_name_opposite + "_" + "_".join(capability_first_words)
##############    
        res1_df = pd.concat([res1_df,
                pd.DataFrame({
                    stability_column_name_modified: stress_column,
                    stability_column_name_opposite_modified: stress_column_opposite,
                })
            ], axis=1)
        
    normalized_data_1 = res1_df.copy()
    min_value = normalized_data_1.min().min()
    max_value = normalized_data_1.max().max()
    normalized_data_1 = 1 - (normalized_data_1 - min_value) / (max_value - min_value)
    
    for column in normalized_data_1.columns:
        existing_data[column] = normalized_data_1[column]
        
    color_palette = ['red', 'blue', 'green', 'orange', 'purple', 'violet', 'yellow']
    color_cycle = cycle(color_palette)
    capability_color_map = {cap_col: next(color_cycle) for cap_col in capability_vector}
    fig = make_subplots(rows=len(change_percentages), cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.1, horizontal_spacing=0.1, row_heights=[0.5] * len(change_percentages))
    
    for idx, intervention in enumerate(change_percentages):
        for cap_col in capability_vector:
            color = capability_color_map[cap_col]

            fig.update_xaxes(title_text=f'Stability ({base_column} +{intervention}%)', row=idx + 1, col=1, showline=True, showgrid=True)
            fig.update_yaxes(title_text='Percent Change', row=idx + 1, col=1, showline=True, showgrid=True)
            fig.update_xaxes(title_text=f'Stability ({base_column} -{intervention}%)', row=idx + 1, col=2, showline=True, showgrid=True)
            fig.update_yaxes(title_text='Percent Change', row=idx + 3, col=2, showline=True, showgrid=True)
            
            fig.update_layout(margin=dict(t=50))
            
            scatter_increase = go.Scatter(
                x=existing_data[stability_column_name_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} +{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=5, symbol='circle', color=color),
                name=f'{cap_col} +{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change: %{y:.2f}' + '<br>Stability: %{x:.2f}',
                showlegend=False
            )

            fig.add_trace(scatter_increase, row=idx + 1, col=1)

            scatter_decrease = go.Scatter(
                x=existing_data[stability_column_name_opposite_modified],
                y=existing_data[
                    f"Percent Change in {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=5, symbol='circle', color=color),
                name=f'{cap_col} -{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change: %{y:.2f}' + '<br>Stability: %{x:.2f}',
                showlegend=False
            )

            fig.add_trace(scatter_decrease, row=idx + 1, col=2)

        avg_stability_increase = existing_data[stability_column_name_modified].mean()
        avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
        all_values = [
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].min(),
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].min(),
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].max(),
            existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].max()
        ]

        filtered_values = [value for value in all_values if not np.isinf(value)]

        # Calculate the range
        if filtered_values:
            y_range_1 = max(abs(min(filtered_values)), max(filtered_values))
        else:
            y_range_1 = 0  # If all values are inf or -inf, set the range to 0

        y_axis_1 = max(y_range_1, 1)
        print(y_axis_1)
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_increase,
                x1=avg_stability_increase,
                y0=-10000,
                y1=y_axis_1,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Stability"
            ),
            row=idx + 1, col=1
        )
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_decrease,
                x1=avg_stability_decrease,
                y0=-10000,
                y1=y_axis_1,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Stability"
            ),
            row=idx + 1, col=2
        )

        avg_impact_increase = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].mean()
        avg_impact_decrease = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

        # Add shapes for average impact lines
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_increase,
                y1=avg_impact_increase,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Impact"
            ),
            row=idx + 1, col=1
        )
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_decrease,
                y1=avg_impact_decrease,
                line=dict(color="black", dash="dash", width=1),
                name="Avg Impact"
            ),
            row=idx + 1, col=2
        )


    for cap_col in capability_vector:
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(size=5, symbol='circle', color=capability_color_map[cap_col]), name=cap_col), row=1, col=1)

    fig.update_layout(
        height=800,  
        showlegend=True,
        legend=dict(title="Capability", orientation="v", yanchor="bottom", y=0.85, xanchor="left", x=1),
    )
    fig.show()

    return existing_data, result_df, res1_df

base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df,res1_df = calc_and_vis_impact_stability_2D1(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

267769.6894546854
291251.27339239337


,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.973840,68479.019211,57507.638667,0.102158,0.081891,1928.406967,1603.425316,66324.632613,54367.859761,...,72303.019039,48389.473335,0.109221,0.065047,2411.393506,1580.773648,0.959922,0.968503,0.955632,0.972793
1,BENGALURU(R),0.983965,57992.196208,48927.510755,0.082786,0.066041,2363.559737,1978.483889,54895.937738,45017.109409,...,59835.351903,40077.695244,0.086190,0.049693,40.514646,-5.883345,0.975871,0.981552,0.973031,0.984393
2,RAMANAGARA,0.868345,27479.191334,23962.324950,0.026420,0.019924,126.576446,97.578537,21642.413549,17809.680527,...,23558.780060,15893.314016,0.019178,0.005018,125.442871,52.089129,0.834974,0.866272,0.819324,0.881922
3,CHITRADURGA,0.477695,101029.330849,84139.711825,0.162286,0.131087,2112.644127,1742.744455,101798.443858,83391.887143,...,111001.722216,74188.608786,0.180708,0.112705,-59.186792,-72.722269,0.694675,0.751482,0.666271,0.779886
4,DAVANAGERE,0.169111,266084.831886,219185.121765,0.467187,0.380551,-45.642060,-55.223109,281678.371090,230566.373060,...,307234.370105,205010.374046,0.543200,0.354366,-48.237305,-65.459953,0.805569,0.842214,0.787247,0.860537


In [33]:
new_data_1

,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.973840,68479.019211,57507.638667,0.102158,0.081891,1928.406967,1603.425316,66324.632613,54367.859761,...,72303.019039,48389.473335,0.109221,0.065047,2.411394e+03,1.580774e+03,0.959922,0.968503,0.955632,0.972793
1,BENGALURU(R),0.983965,57992.196208,48927.510755,0.082786,0.066041,2363.559737,1978.483889,54895.937738,45017.109409,...,59835.351903,40077.695244,0.086190,0.049693,4.051465e+01,-5.883345e+00,0.975871,0.981552,0.973031,0.984393
2,RAMANAGARA,0.868345,27479.191334,23962.324950,0.026420,0.019924,126.576446,97.578537,21642.413549,17809.680527,...,23558.780060,15893.314016,0.019178,0.005018,1.254429e+02,5.208913e+01,0.834974,0.866272,0.819324,0.881922
3,CHITRADURGA,0.477695,101029.330849,84139.711825,0.162286,0.131087,2112.644127,1742.744455,101798.443858,83391.887143,...,111001.722216,74188.608786,0.180708,0.112705,-5.918679e+01,-7.272227e+01,0.694675,0.751482,0.666271,0.779886
4,DAVANAGERE,0.169111,266084.831886,219185.121765,0.467187,0.380551,-45.642060,-55.223109,281678.371090,230566.373060,...,307234.370105,205010.374046,0.543200,0.354366,-4.823731e+01,-6.545995e+01,0.805569,0.842214,0.787247,0.860537
5,KOLAR,0.918093,65579.675404,55135.448279,0.096802,0.077509,7490.240209,6281.417625,63164.884832,51782.611577,...,68856.021460,46091.474949,0.102854,0.060802,3.814498e+03,2.520323e+03,0.997331,0.999110,0.996442,1.000000
6,CHIKKABALLAPURA,0.883837,81544.187243,68197.321602,0.126292,0.101637,2922.393893,2427.699096,80563.246050,66017.634391,...,87836.051879,58744.828562,0.137915,0.084176,-1.673598e+01,-4.431295e+01,0.994820,0.997056,0.993702,0.998173
7,SHIVAMOGGA,0.630729,154680.134786,128035.824137,0.261393,0.212174,-59.820628,-66.741696,160267.883819,131230.519838,...,174786.565809,116711.837848,0.298535,0.191256,-2.284244e+01,-4.847887e+01,0.967931,0.975056,0.964369,0.978618
8,TUMAKURU,0.865884,106890.147260,88934.925252,0.173113,0.139945,310.452912,241.505742,108185.648301,88617.781687,...,117969.581608,78833.848381,0.193579,0.121286,7.642158e+01,1.789473e+01,0.987103,0.990742,0.985284,0.992561
9,CHIKKAMAGALURU,0.698987,183813.362651,151872.101481,0.315210,0.256206,66.297271,37.399782,192017.706609,157207.647575,...,209422.736125,139802.618059,0.362517,0.233911,2.413407e+02,1.278660e+02,0.861176,0.887711,0.847909,0.900978


In [34]:
# new_data_1[['New Stability(NPK + 10%)_Rice_Jowar', 'New Stability(NPK - 10%)_Rice_Jowar', 'New Stability(NPK + 20%)_Rice_Jowar', 'New Stability(NPK - 20%)_Rice_Jowar']]

KeyError: "None of [Index(['New Stability(NPK + 10%)_Rice_Jowar',\n       'New Stability(NPK - 10%)_Rice_Jowar',\n       'New Stability(NPK + 20%)_Rice_Jowar',\n       'New Stability(NPK - 20%)_Rice_Jowar'],\n      dtype='object')] are in the [columns]"

In [35]:
res1_df

,New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,26813.859929,21938.612669,29251.483559,19500.989040
1,17752.360442,14524.658544,19366.211391,12910.807594
2,97804.157146,80021.583119,106695.444159,71130.296106
3,177515.946601,145240.319946,193653.759928,129102.506619
4,114510.459251,93690.375751,124920.501001,83280.334001
5,5559.577983,4548.745623,6064.994164,4043.329442
6,6986.485280,5716.215229,7621.620306,5081.080204
7,22263.582742,18215.658607,24287.544810,16191.696540
8,11370.770082,9303.357340,12404.476454,8269.650969
9,82917.243892,67841.381366,90455.175155,60303.450103


In [36]:
normalized_data_1 = res1_df.copy()
min_value = normalized_data_1.min().min()
max_value = normalized_data_1.max().max()
normalized_data_1 = (normalized_data_1 - min_value) / (max_value - min_value)
normalized_data_2 = 1 - (normalized_data_1 - min_value) / (max_value - min_value)

In [37]:
normalized_data_1

,New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,0.040078,0.031497,0.044368,0.027207
1,0.024129,0.018448,0.026969,0.015607
2,0.165026,0.133728,0.180676,0.118078
3,0.305325,0.248518,0.333729,0.220114
4,0.194431,0.157786,0.212753,0.139463
5,0.002669,0.000890,0.003558,0.000000
6,0.005180,0.002944,0.006298,0.001827
7,0.032069,0.024944,0.035631,0.021382
8,0.012897,0.009258,0.014716,0.007439
9,0.138824,0.112289,0.152091,0.099022


In [38]:
normalized_data_2

,New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,1.007116,1.007117,1.007116,1.007117
1,1.007117,1.007117,1.007117,1.007117
2,1.007116,1.007116,1.007116,1.007116
3,1.007116,1.007116,1.007116,1.007116
4,1.007116,1.007116,1.007116,1.007116
5,1.007117,1.007117,1.007117,1.007117
6,1.007117,1.007117,1.007117,1.007117
7,1.007117,1.007117,1.007117,1.007117
8,1.007117,1.007117,1.007117,1.007117
9,1.007116,1.007116,1.007116,1.007116


In [ ]:
base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df,res1_df = calc_and_vis_impact_stability_2D1(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

267769.6894546854
291251.27339239337


,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.973840,68479.019211,57507.638667,0.102158,0.081891,1928.406967,1603.425316,66324.632613,54367.859761,...,72303.019039,48389.473335,0.109221,0.065047,2411.393506,1580.773648,0.959922,0.968503,0.955632,0.972793
1,BENGALURU(R),0.983965,57992.196208,48927.510755,0.082786,0.066041,2363.559737,1978.483889,54895.937738,45017.109409,...,59835.351903,40077.695244,0.086190,0.049693,40.514646,-5.883345,0.975871,0.981552,0.973031,0.984393
2,RAMANAGARA,0.868345,27479.191334,23962.324950,0.026420,0.019924,126.576446,97.578537,21642.413549,17809.680527,...,23558.780060,15893.314016,0.019178,0.005018,125.442871,52.089129,0.834974,0.866272,0.819324,0.881922
3,CHITRADURGA,0.477695,101029.330849,84139.711825,0.162286,0.131087,2112.644127,1742.744455,101798.443858,83391.887143,...,111001.722216,74188.608786,0.180708,0.112705,-59.186792,-72.722269,0.694675,0.751482,0.666271,0.779886
4,DAVANAGERE,0.169111,266084.831886,219185.121765,0.467187,0.380551,-45.642060,-55.223109,281678.371090,230566.373060,...,307234.370105,205010.374046,0.543200,0.354366,-48.237305,-65.459953,0.805569,0.842214,0.787247,0.860537


In [ ]:
new_data_1

,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 10%)_Rice_Maize,New Stability(NPK - 10%)_Rice_Maize,New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.973840,68479.019211,57507.638667,0.102158,0.081891,1928.406967,1603.425316,66324.632613,54367.859761,...,72303.019039,48389.473335,0.109221,0.065047,2.411394e+03,1.580774e+03,0.959922,0.968503,0.955632,0.972793
1,BENGALURU(R),0.983965,57992.196208,48927.510755,0.082786,0.066041,2363.559737,1978.483889,54895.937738,45017.109409,...,59835.351903,40077.695244,0.086190,0.049693,4.051465e+01,-5.883345e+00,0.975871,0.981552,0.973031,0.984393
2,RAMANAGARA,0.868345,27479.191334,23962.324950,0.026420,0.019924,126.576446,97.578537,21642.413549,17809.680527,...,23558.780060,15893.314016,0.019178,0.005018,1.254429e+02,5.208913e+01,0.834974,0.866272,0.819324,0.881922
3,CHITRADURGA,0.477695,101029.330849,84139.711825,0.162286,0.131087,2112.644127,1742.744455,101798.443858,83391.887143,...,111001.722216,74188.608786,0.180708,0.112705,-5.918679e+01,-7.272227e+01,0.694675,0.751482,0.666271,0.779886
4,DAVANAGERE,0.169111,266084.831886,219185.121765,0.467187,0.380551,-45.642060,-55.223109,281678.371090,230566.373060,...,307234.370105,205010.374046,0.543200,0.354366,-4.823731e+01,-6.545995e+01,0.805569,0.842214,0.787247,0.860537
5,KOLAR,0.918093,65579.675404,55135.448279,0.096802,0.077509,7490.240209,6281.417625,63164.884832,51782.611577,...,68856.021460,46091.474949,0.102854,0.060802,3.814498e+03,2.520323e+03,0.997331,0.999110,0.996442,1.000000
6,CHIKKABALLAPURA,0.883837,81544.187243,68197.321602,0.126292,0.101637,2922.393893,2427.699096,80563.246050,66017.634391,...,87836.051879,58744.828562,0.137915,0.084176,-1.673598e+01,-4.431295e+01,0.994820,0.997056,0.993702,0.998173
7,SHIVAMOGGA,0.630729,154680.134786,128035.824137,0.261393,0.212174,-59.820628,-66.741696,160267.883819,131230.519838,...,174786.565809,116711.837848,0.298535,0.191256,-2.284244e+01,-4.847887e+01,0.967931,0.975056,0.964369,0.978618
8,TUMAKURU,0.865884,106890.147260,88934.925252,0.173113,0.139945,310.452912,241.505742,108185.648301,88617.781687,...,117969.581608,78833.848381,0.193579,0.121286,7.642158e+01,1.789473e+01,0.987103,0.990742,0.985284,0.992561
9,CHIKKAMAGALURU,0.698987,183813.362651,151872.101481,0.315210,0.256206,66.297271,37.399782,192017.706609,157207.647575,...,209422.736125,139802.618059,0.362517,0.233911,2.413407e+02,1.278660e+02,0.861176,0.887711,0.847909,0.900978


In [ ]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, change_percentages, dim):
    min_value = float('inf')
    max_value = float('-inf')
    
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    col_pass_plus = []
    col_pass_minus = []
    result_df = pd.DataFrame()
    
    for intervention in change_percentages:
        for cap_col in capability_vector:
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            m, c = model.params[base_column], model.params['const']

            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())
            
    
    result_df = pd.DataFrame()
    res1_df = pd.DataFrame()
    
    for intervention in change_percentages:
        col_pass_plus = []
        col_pass_minus = []
        for cap_col in capability_vector:
            # Calculate Impact Score
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']
            # print(m,c)
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )*100/(y)
            percent_change_dec = (new_vector_decrease - (y) )*100/(y)

            result_df = pd.concat([result_df,
                pd.DataFrame({
                    f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
                    f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
                    f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
                    f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
                    f'Percent Change in {cap_col} ({base_column} +{intervention}%)': percent_change_inc,
                    f'Percent Change in {cap_col} ({base_column} -{intervention}%)': percent_change_dec,
                    
                })
            ], axis=1)

            columns_to_pass = f'{cap_col} ({base_column} +{intervention}%)'
            col_pass_plus.append(columns_to_pass)
            
            # Calculate Stability for the opposite intervention and add to DataFrame
            opposite_intervention = -intervention
            columns_to_pass_opposite = f'{cap_col} ({base_column} -{intervention}%)'
            col_pass_minus.append(columns_to_pass_opposite)
        
        for column in result_df.columns:
            existing_data[column] = result_df[column]

        # print(col_pass_minus)
        # print(col_pass_plus)
        
        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

        normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())

        capability_first_words = [col.split('_')[0] for col in capability_vector]
        stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_modified] = normalized_stability_vector
        
        stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

        # Normalize stability values for the opposite intervention
        normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
        stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
        existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
        
        res1_df = pd.concat([res1_df,
            pd.DataFrame({
            stability_column_name_modified: stability_column,
            stability_column_name_opposite_modified: stability_column_opposite,
            })
        ], axis=1)
        
        normalized_data_1 = res1_df.copy()
        min_value = normalized_data_1.min().min()
        max_value = normalized_data_1.max().max()
        normalized_data_1 = (normalized_data_1 - min_value) / (max_value - min_value)
        
        # for col in normalized_data_1.columns:
        for column in normalized_data_1.columns:
            existing_data[column] = normalized_data_1[column]
        
        # Visualize with Plotly for +intervention
        fig_increase = go.Figure()
        color_palette = ['red', 'blue', 'green', 'orange', 'purple'] 

        for i, cap_col in enumerate(capability_vector):
            symbol = 'star' if i == 0 else 'triangle-up' 
            scatter_increase = go.Scatter(
                x=existing_data[stability_column_name_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} +{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7, symbol=symbol, color=color_palette[i]),  
                name=f'{cap_col} +{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change in: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )
            fig_increase.add_trace(scatter_increase)
        
        # Add average lines
        # print("aefawefaw")
        avg_stability_increase = existing_data[stability_column_name_modified].mean()
        avg_impact_increase = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()
        y_range_1 = max(abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].min()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].min()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} +{abs(intervention)}%)"].max()), 
                    abs(existing_data[f"Percent Change in {capability_vector[0]} ({base_column} -{abs(intervention)}%)"].max()))
        y_axis_1 = max(y_range_1,1)


        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_increase,
                x1=avg_stability_increase,
                y0=0,
                y1=y_axis_1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )
        fig_increase.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_increase,
                y1=avg_impact_increase,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for +intervention plot
        fig_increase.update_layout(
            title=f'Percent Change vs Stability (+{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
            yaxis=dict(title='Percent Change'),
            hovermode='closest'
        )

        # Visualize with Plotly for -intervention
        fig_decrease = go.Figure()

        for i, cap_col in enumerate(capability_vector):
            # Scatter plot for -intervention
            symbol = 'star' if i == 0 else 'triangle-up'
            
            scatter_decrease = go.Scatter(
                x=existing_data[stability_column_name_opposite_modified],
                y=existing_data[f"Percent Change in {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
                mode='markers',
                marker=dict(size=7,symbol=symbol, color=color_palette[i]),
                name=f'{cap_col} -{intervention}%',
                text=existing_data['District'],
                hovertemplate='%{text}' + '<br>Percent Change: %{y:.2f}' + '<br>Stability: %{x:.2f}'
            )
            
            fig_decrease.add_trace(scatter_decrease)

        # Add average lines for -intervention
        avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
        avg_impact_decrease = existing_data[f"Percent Change in {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=avg_stability_decrease,
                x1=avg_stability_decrease,
                y0=0,
                y1=y_axis_1,
                line=dict(color="black", dash="dash"),
                name="Average Stability"
            )
        )

        fig_decrease.add_shape(
            go.layout.Shape(
                type="line",
                x0=0,
                x1=1,
                y0=avg_impact_decrease,
                y1=avg_impact_decrease,
                line=dict(color="black", dash="dash"),
                name="Average Impact"
            )
        )

        # Update layout for -intervention plot
        fig_decrease.update_layout(
            title=f'Percent Change vs Stability (-{intervention}%)',
            xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
            yaxis=dict(title='Percent Change'),
            hovermode='closest'
        )


        fig_increase.show()
        fig_decrease.show()

    return existing_data, result_df

base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
# change_percentage = 20
change_percentages = [10,20]
dim = len(capability_vector)

new_data_2,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_2.head()

,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Percent Change in Rice_Production (TotalNPK +20%),Percent Change in Rice_Production (TotalNPK -20%),Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.973840,68479.019211,57507.638667,0.102158,0.081891,1928.406967,1603.425316,66324.632613,54367.859761,...,2090.897793,1440.934490,72303.019039,48389.473335,0.109221,0.065047,2411.393506,1580.773648,0.955632,0.972793
1,BENGALURU(R),0.983965,57992.196208,48927.510755,0.082786,0.066041,2363.559737,1978.483889,54895.937738,45017.109409,...,2556.097661,1785.945966,59835.351903,40077.695244,0.086190,0.049693,40.514646,-5.883345,0.973031,0.984393
2,RAMANAGARA,0.868345,27479.191334,23962.324950,0.026420,0.019924,126.576446,97.578537,21642.413549,17809.680527,...,141.075400,83.079582,23558.780060,15893.314016,0.019178,0.005018,125.442871,52.089129,0.819324,0.881922
3,CHITRADURGA,0.477695,101029.330849,84139.711825,0.162286,0.131087,2112.644127,1742.744455,101798.443858,83391.887143,...,2297.593963,1557.794619,111001.722216,74188.608786,0.180708,0.112705,-59.186792,-72.722269,0.666271,0.779886
4,DAVANAGERE,0.169111,266084.831886,219185.121765,0.467187,0.380551,-45.642060,-55.223109,281678.371090,230566.373060,...,-40.851536,-60.013633,307234.370105,205010.374046,0.543200,0.354366,-48.237305,-65.459953,0.787247,0.860537


In [40]:
import statsmodels.api as sm
import pandas as pd
import plotly.graph_objects as go

def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, change_percentages, dim):
    # Calculate min and max values for changes made by each column in capability_vector
    min_value = float('inf')
    max_value = float('-inf')
    
    change_min_value = float('inf')
    change_max_value = float('-inf')
    
    initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
    existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

    col_pass_plus = []
    col_pass_minus = []
    result_df = pd.DataFrame()

    # Create fixed columns
    fixed_columns = {
        'District': [],
        'Initial_Stability': [],
        'Base_Column': [],
        'Intervention_Percentage': [],
        'Capability_Vector': [],
        'New_Values': [],
        'Normalized_New_Values': [],
        'Percent_Change': []
    }
    
    for intervention in change_percentages:
        for cap_col in capability_vector:
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()
            print(cap_col)            
            m, c = model.params[base_column], model.params['const']
            print(m,c)
            
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            change_vector_increase = m * base_column_increase + c
            change_vector_decrease = m * base_column_decrease + c
            
            change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
            change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())

    for intervention in change_percentages:
        for cap_col in capability_vector:
            X = sm.add_constant(df[base_column])
            y = df[cap_col]
            model = sm.OLS(y, X).fit()

            m, c = model.params[base_column], model.params['const']
            base_column_increase = (1 + intervention/100) * df[base_column]
            base_column_decrease = (1 - intervention/100) * df[base_column]

            new_vector_increase = m * base_column_increase + c
            new_vector_decrease = m * base_column_decrease + c
            
            normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
            normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
            percent_change_inc = (new_vector_increase - (y) )*100/(y)
            percent_change_dec = (new_vector_decrease - (y) )*100/(y)

            fixed_columns['District'].extend(existing_data['District'].tolist())
            fixed_columns['Initial_Stability'].extend(existing_data['Initial Stability'].tolist())
            fixed_columns['Base_Column'].extend([base_column] * len(existing_data))
            fixed_columns['Intervention_Percentage'].extend([intervention] * len(existing_data))
            fixed_columns['Capability_Vector'].extend([cap_col] * len(existing_data))
            fixed_columns['New_Values'].extend(new_vector_increase.tolist())
            fixed_columns['Normalized_New_Values'].extend(normalized_new_increase_vector.tolist())
            fixed_columns['Percent_Change'].extend(percent_change_inc.tolist())

            fixed_columns['District'].extend(existing_data['District'].tolist())
            fixed_columns['Initial_Stability'].extend(existing_data['Initial Stability'].tolist())
            fixed_columns['Base_Column'].extend([base_column] * len(existing_data))
            fixed_columns['Intervention_Percentage'].extend([-intervention] * len(existing_data))
            fixed_columns['Capability_Vector'].extend([cap_col] * len(existing_data))
            fixed_columns['New_Values'].extend(new_vector_decrease.tolist())
            fixed_columns['Normalized_New_Values'].extend(normalized_new_decrease_vector.tolist())
            fixed_columns['Percent_Change'].extend(percent_change_dec.tolist())
            
    # Create DataFrame from fixed columns
    result_df = pd.DataFrame(fixed_columns)

    return result_df

# Usage
base_column = "TotalNPK"
capability_vector = ["Rice_Production", "Maize_Production"]
change_percentages = [10, 20]
dim = len(capability_vector)

result_df_3 = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
result_df_3.head()


Rice_Production
2.353363480056113 8136.426218911565
Maize_Production
2.564730341506419 562.38192631683
Rice_Production
2.353363480056113 8136.426218911565
Maize_Production
2.564730341506419 562.38192631683


,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change
0,BENGALURU,0.973840,TotalNPK,10,Rice_Production,68479.019211,0.102158,1928.406967
1,BENGALURU(R),0.983965,TotalNPK,10,Rice_Production,57992.196208,0.082786,2363.559737
2,RAMANAGARA,0.868345,TotalNPK,10,Rice_Production,27479.191334,0.026420,126.576446
3,CHITRADURGA,0.477695,TotalNPK,10,Rice_Production,101029.330849,0.162286,2112.644127
4,DAVANAGERE,0.169111,TotalNPK,10,Rice_Production,266084.831886,0.467187,-45.642060


In [87]:
result_df_3

,District,Initial_Stability,Base_Column,Intervention_Percentage,Capability_Vector,New_Values,Normalized_New_Values,Percent_Change
0,BENGALURU,0.973840,TotalNPK,10,Rice_Production,68479.019211,0.102158,1928.406967
1,BENGALURU(R),0.983965,TotalNPK,10,Rice_Production,57992.196208,0.082786,2363.559737
2,RAMANAGARA,0.868345,TotalNPK,10,Rice_Production,27479.191334,0.026420,126.576446
3,CHITRADURGA,0.477695,TotalNPK,10,Rice_Production,101029.330849,0.162286,2112.644127
4,DAVANAGERE,0.169111,TotalNPK,10,Rice_Production,266084.831886,0.467187,-45.642060
...,...,...,...,...,...,...,...,...
235,BIDAR,0.976793,TotalNPK,-20,Maize_Production,51155.278535,0.070156,1732.865587
236,KALABURAGI,0.826587,TotalNPK,-20,Maize_Production,111356.680895,0.181364,661.621509
237,YADGIRI,0.823837,TotalNPK,-20,Maize_Production,188072.894870,0.323078,6936.023003
238,RAICHUR,0.293136,TotalNPK,-20,Maize_Production,241103.311196,0.421039,33247.622572


In [88]:
# import statsmodels.api as sm
# import pandas as pd
# import plotly.graph_objects as go

# def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, change_percentages, dim):
#     min_value = float('inf')
#     max_value = float('-inf')
    
#     change_min_value = float('inf')
#     change_max_value = float('-inf')
    
#     initial_stability_column = calculate_and_return_initial_stability(df, capability_vector, G)
#     existing_data = pd.concat([existing_data, initial_stability_column], axis=1)

#     col_pass_plus = []
#     col_pass_minus = []
#     result_df = pd.DataFrame()

#     # Create fixed columns
#     fixed_columns = {
#         'District': [],
#         'Initial_Stability': [],
#         'Base_Column': [],
#         'Intervention_Percentage': [],
#         'Capability_Vector': [],
#         'New_Values': [],
#         'Normalized_New_Values': [],
#         'Percent_Change': [],
#         'Stability': []  # Added stability column
#     }
    
#     for intervention in change_percentages:
#         for cap_col in capability_vector:
#             X = sm.add_constant(df[base_column])
#             y = df[cap_col]
#             model = sm.OLS(y, X).fit()
#             m, c = model.params[base_column], model.params['const']

#             base_column_increase = (1 + intervention/100) * df[base_column]
#             base_column_decrease = (1 - intervention/100) * df[base_column]

#             change_vector_increase = m * base_column_increase + c
#             change_vector_decrease = m * base_column_decrease + c
            
#             change_min_value = min(change_min_value, change_vector_increase.min(), change_vector_decrease.min())
#             change_max_value = max(change_max_value, change_vector_increase.max(), change_vector_decrease.max())

#     for intervention in change_percentages:
#         for cap_col in capability_vector:
#             X = sm.add_constant(df[base_column])
#             y = df[cap_col]
#             model = sm.OLS(y, X).fit()

#             m, c = model.params[base_column], model.params['const']
#             base_column_increase = (1 + intervention/100) * df[base_column]
#             base_column_decrease = (1 - intervention/100) * df[base_column]

#             new_vector_increase = m * base_column_increase + c
#             new_vector_decrease = m * base_column_decrease + c
            
#             normalized_new_increase_vector = (new_vector_increase - change_min_value) / (change_max_value - change_min_value)
#             normalized_new_decrease_vector = (new_vector_decrease - change_min_value) / (change_max_value - change_min_value)
            
#             percent_change_inc = (new_vector_increase - (y) )*100/(y)
#             percent_change_dec = (new_vector_decrease - (y) )*100/(y)

#             # Calculate stability here
#             print("aawefaw")
#             stability_column_plus, _ = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, new_vector_increase.tolist(), intervention, dim)
#             stability_column_minus, _ = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, new_vector_decrease.tolist(), intervention, dim)
          
#             fixed_columns['District'].extend(existing_data['District'].tolist())
#             fixed_columns['Initial_Stability'].extend(existing_data['Initial Stability'].tolist())
#             fixed_columns['Base_Column'].extend([base_column] * len(existing_data))
#             fixed_columns['Intervention_Percentage'].extend([intervention] * len(existing_data))
#             fixed_columns['Capability_Vector'].extend([cap_col] * len(existing_data))
#             fixed_columns['New_Values'].extend(new_vector_increase.tolist())
#             fixed_columns['Normalized_New_Values'].extend(normalized_new_increase_vector.tolist())
#             fixed_columns['Percent_Change'].extend(percent_change_inc.tolist())
#             fixed_columns['Stability'].extend(stability_column_plus.tolist())  # Add stability values

#             fixed_columns['District'].extend(existing_data['District'].tolist())
#             fixed_columns['Initial_Stability'].extend(existing_data['Initial Stability'].tolist())
#             fixed_columns['Base_Column'].extend([base_column] * len(existing_data))
#             fixed_columns['Intervention_Percentage'].extend([-intervention] * len(existing_data))
#             fixed_columns['Capability_Vector'].extend([cap_col] * len(existing_data))
#             fixed_columns['New_Values'].extend(new_vector_decrease.tolist())
#             fixed_columns['Normalized_New_Values'].extend(normalized_new_decrease_vector.tolist())
#             fixed_columns['Percent_Change'].extend(percent_change_dec.tolist())
#             fixed_columns['Stability'].extend(stability_column_minus.tolist())  # Add stability values
            
#     # Create DataFrame from fixed columns
#     result_df = pd.DataFrame(fixed_columns)

#     return result_df

# # Usage
# base_column = "TotalNPK"
# capability_vector = ["Rice_Production", "Maize_Production"]
# change_percentages = [10, 20]
# dim = len(capability_vector)

# result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
# result_df.head()


In [94]:
Stability_column_values = pd.concat([new_data_1['New Stability(NPK + 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK + 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 10%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK + 20%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 20%)_Rice_Maize'],
                               new_data_1['New Stability(NPK + 20%)_Rice_Maize'], 
                               new_data_1['New Stability(NPK - 20%)_Rice_Maize']], axis=0)

Stability_column_values.reset_index(drop=True, inplace=True)

result_df_3['Stability'] = Stability_column_values

In [89]:
result_df_3.to_excel('Rice_Maize_10_20_1.xlsx', index =False)

In [44]:
new_data_2.to_excel('Rice_Maize_10_20.xlsx', index =False)

In [194]:
# import statsmodels.api as sm
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go

# def calc_and_vis_impact_stability(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
#     # Calculate min and max values for changes made by each column in capability_vector
#     min_value_increase = float('inf')
#     max_value_increase = float('-inf')
#     min_value_decrease = float('inf')
#     max_value_decrease = float('-inf')

#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         change_vector_increase = m * base_column_increase + c
#         change_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = change_vector_increase - m * df[base_column] - c  
#         vector_minus = change_vector_decrease - m * df[base_column] - c

#         # Update min and max values for each column's changes
#         min_value_increase = min(min_value_increase, vector_plus.min())
#         max_value_increase = max(max_value_increase, vector_plus.max())

#         min_value_decrease = min(min_value_decrease, vector_minus.min())
#         max_value_decrease = max(max_value_decrease, vector_minus.max())

#     # Normalize changes manually using the calculated min and max values
#     col_pass_plus = []
#     col_pass_minus = []
#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         new_vector_increase = m * base_column_increase + c
#         new_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = new_vector_increase - m * df[base_column] - c  
#         vector_minus = new_vector_decrease - m * df[base_column] - c

#         # Normalize changes manually using the calculated min and max values
#         normalized_change_increase_vector = (vector_plus - min_value_increase) / (max_value_increase - min_value_increase)
#         normalized_change_decrease_vector = (vector_minus - min_value_decrease) / (max_value_decrease - min_value_decrease)

#         scaler = MinMaxScaler()
#         normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))
#         normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))
        
#         result_df = pd.DataFrame({
#             f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
#             f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
#             f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
#             f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
#             f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
#             f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,

#         })

#         # Merge Columns
#         for column in result_df.columns:
#             if column in result_df.columns:
#                 existing_data[column] = result_df[column]

#         # Calculate Stability and add to DataFrame
#         columns_to_pass = f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
#         col_pass_plus.append(columns_to_pass)
        
#         # Calculate Stability for the opposite intervention and add to DataFrame
#         opposite_intervention = -intervention
#         columns_to_pass_opposite = f"Normalized {cap_col} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"
#         col_pass_minus.append(columns_to_pass_opposite)
        
#     # Normalize stability values
#     stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

#     normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())
        
#     capability_first_words = [col.split('_')[0] for col in capability_vector]
#     stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
#     existing_data[stability_column_name_modified] = normalized_stability_vector
    
#     stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)
        
#     # Normalize stability values for the opposite intervention
#     normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
#     stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
#     existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
    
#     # Visualize with Plotly for +intervention
#     fig_increase = go.Figure()
#     color_palette = ['red', 'blue', 'green', 'orange', 'purple']  # Add more colors if needed

#     for i, cap_col in enumerate(capability_vector):
#         symbol = 'star' if i == 0 else 'circle'  # Use 'star' for the first point, 'circle' for others
#         scatter_increase = go.Scatter(
#             x=existing_data[stability_column_name_modified],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=7, symbol=symbol, color=color_palette[i]),  # Adjust the size and add symbol parameter
#             name=f'{cap_col} +{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )
#         fig_increase.add_trace(scatter_increase)

#     # Add average lines
#     avg_stability_increase = existing_data[stability_column_name_modified].mean()
#     avg_impact_increase = existing_data[f"Normalized {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()

#     fig_increase.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=avg_stability_increase,
#             x1=avg_stability_increase,
#             y0=0,
#             y1=1,
#             line=dict(color="black", dash="dash"),
#             name="Average Stability"
#         )
#     )
#     fig_increase.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=0,
#             x1=1,
#             y0=avg_impact_increase,
#             y1=avg_impact_increase,
#             line=dict(color="black", dash="dash"),
#             name="Average Impact"
#         )
#     )

#     # Update layout for +intervention plot
#     fig_increase.update_layout(
#         title=f'Impact vs Stability (+{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Visualize with Plotly for -intervention
#     fig_decrease = go.Figure()

#     for i, cap_col in enumerate(capability_vector):
#         # Scatter plot for -intervention
#         symbol = 'star' if i == 0 else 'circle'
#         scatter_decrease = go.Scatter(
#             x=existing_data[stability_column_name_opposite_modified],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=7,symbol=symbol, color=color_palette[i]),
#             name=f'{cap_col} -{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )

#         fig_decrease.add_trace(scatter_decrease)

#     # Add average lines for -intervention
#     avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
#     avg_impact_decrease = existing_data[f"Normalized {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()

#     fig_decrease.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=avg_stability_decrease,
#             x1=avg_stability_decrease,
#             y0=0,
#             y1=1,
#             line=dict(color="black", dash="dash"),
#             name="Average Stability"
#         )
#     )
#     fig_decrease.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=0,
#             x1=1,
#             y0=avg_impact_decrease,
#             y1=avg_impact_decrease,
#             line=dict(color="black", dash="dash"),
#             name="Average Impact"
#         )
#     )

#     # Update layout for -intervention plot
#     fig_decrease.update_layout(
#         title=f'Impact vs Stability (-{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Show plots
#     fig_increase.show()
#     fig_decrease.show()

#     return existing_data, fig_increase, fig_decrease

# base_column = "TotalNPK_315"
# capability_vector = ["Rice_Production_223", "Jowar_Yield_278"]
# change_percentage = 20
# dim = 2

# new_data, handle_increase, handle_decrease = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
# new_data.head()

In [48]:
result_df

,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),Normalized Maize_Production (TotalNPK +10%),Normalized Maize_Production (TotalNPK -10%),...,Normalized Rice_Production (TotalNPK +20%),Normalized Rice_Production (TotalNPK -20%),Percent Change in Rice_Production (TotalNPK +20%),Percent Change in Rice_Production (TotalNPK -20%),Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%)
0,68479.019211,57507.638667,0.102158,0.081891,1928.406967,1603.425316,66324.632613,54367.859761,0.098178,0.076091,...,0.112291,0.071757,2090.897793,1440.934490,72303.019039,48389.473335,0.109221,0.065047,2.411394e+03,1.580774e+03
1,57992.196208,48927.510755,0.082786,0.066041,2363.559737,1978.483889,54895.937738,45017.109409,0.077066,0.058817,...,0.091158,0.057668,2556.097661,1785.945966,59835.351903,40077.695244,0.086190,0.049693,4.051465e+01,-5.883345e+00
2,27479.191334,23962.324950,0.026420,0.019924,126.576446,97.578537,21642.413549,17809.680527,0.015638,0.008558,...,0.029669,0.016675,141.075400,83.079582,23558.780060,15893.314016,0.019178,0.005018,1.254429e+02,5.208913e+01
3,101029.330849,84139.711825,0.162286,0.131087,2112.644127,1742.744455,101798.443858,83391.887143,0.163707,0.129706,...,0.177886,0.115487,2297.593963,1557.794619,111001.722216,74188.608786,0.180708,0.112705,-5.918679e+01,-7.272227e+01
4,266084.831886,219185.121765,0.467187,0.380551,-45.642060,-55.223109,281678.371090,230566.373060,0.495992,0.401575,...,0.510505,0.337233,-40.851536,-60.013633,307234.370105,205010.374046,0.543200,0.354366,-4.823731e+01,-6.545995e+01
5,65579.675404,55135.448279,0.096802,0.077509,7490.240209,6281.417625,63164.884832,51782.611577,0.092341,0.071315,...,0.106448,0.067862,8094.651501,5677.006333,68856.021460,46091.474949,0.102854,0.060802,3.814498e+03,2.520323e+03
6,81544.187243,68197.321602,0.126292,0.101637,2922.393893,2427.699096,80563.246050,66017.634391,0.124480,0.097611,...,0.138620,0.089310,3169.741292,2180.351697,87836.051879,58744.828562,0.137915,0.084176,-1.673598e+01,-4.431295e+01
7,154680.134786,128035.824137,0.261393,0.212174,-59.820628,-66.741696,160267.883819,131230.519838,0.271715,0.218076,...,0.286003,0.187565,-56.360094,-70.202230,174786.565809,116711.837848,0.298535,0.191256,-2.284244e+01,-4.847887e+01
8,106890.147260,88934.925252,0.173113,0.139945,310.452912,241.505742,108185.648301,88617.781687,0.175506,0.139359,...,0.189697,0.123361,344.926497,207.032157,117969.581608,78833.848381,0.193579,0.121286,7.642158e+01,1.789473e+01
9,183813.362651,151872.101481,0.315210,0.256206,66.297271,37.399782,192017.706609,157207.647575,0.330365,0.266062,...,0.344712,0.226704,80.746015,22.951038,209422.736125,139802.618059,0.362517,0.233911,2.413407e+02,1.278660e+02


In [49]:
print(result_df['Rice_Production (TotalNPK +20%)'].min(), result_df['Rice_Production (TotalNPK +20%)'].max())

25498.600629373545 516440.3456216234


In [51]:
print(result_df['Maize_Production (TotalNPK +20%)'].min(), result_df['Maize_Production (TotalNPK +20%)'].max())

19483.936493814585 554519.5142804249


In [ ]:
capability_vector = ["Rice_Production", "Maize_Production"]
change_percentages = [10,20]
dim = len(capability_vector)

new_data_1,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_1.head()

In [57]:
base_column = "TotalNPK"
capability_vector = ["Rice_Production","Maize_Production"]
# change_percentage = 20
change_percentages= [10,20]
dim = len(capability_vector)

new_data, result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data.head()

,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Maize_Production (TotalNPK +10%),Maize_Production (TotalNPK -10%),...,Percent Change in Rice_Production (TotalNPK +20%),Percent Change in Rice_Production (TotalNPK -20%),Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 20%)_Rice_Maize,New Stability(NPK - 20%)_Rice_Maize
0,BENGALURU,0.973840,68479.019211,57507.638667,0.102158,0.081891,1928.406967,1603.425316,66324.632613,54367.859761,...,2090.897793,1440.934490,72303.019039,48389.473335,0.109221,0.065047,2411.393506,1580.773648,0.955632,0.972793
1,BENGALURU(R),0.983965,57992.196208,48927.510755,0.082786,0.066041,2363.559737,1978.483889,54895.937738,45017.109409,...,2556.097661,1785.945966,59835.351903,40077.695244,0.086190,0.049693,40.514646,-5.883345,0.973031,0.984393
2,RAMANAGARA,0.868345,27479.191334,23962.324950,0.026420,0.019924,126.576446,97.578537,21642.413549,17809.680527,...,141.075400,83.079582,23558.780060,15893.314016,0.019178,0.005018,125.442871,52.089129,0.819324,0.881922
3,CHITRADURGA,0.477695,101029.330849,84139.711825,0.162286,0.131087,2112.644127,1742.744455,101798.443858,83391.887143,...,2297.593963,1557.794619,111001.722216,74188.608786,0.180708,0.112705,-59.186792,-72.722269,0.666271,0.779886
4,DAVANAGERE,0.169111,266084.831886,219185.121765,0.467187,0.380551,-45.642060,-55.223109,281678.371090,230566.373060,...,-40.851536,-60.013633,307234.370105,205010.374046,0.543200,0.354366,-48.237305,-65.459953,0.787247,0.860537


In [59]:
base_column = "TotalNPK"
capability_vector = [ "Rice_Production","Jowar_Yield","Maize_Production"]
change_percentage = 20
dim = 3

new_data_3d,result_df = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentages, dim)
new_data_3d.head()

,District,Initial Stability,Rice_Production (TotalNPK +10%),Rice_Production (TotalNPK -10%),Normalized Rice_Production (TotalNPK +10%),Normalized Rice_Production (TotalNPK -10%),Percent Change in Rice_Production (TotalNPK +10%),Percent Change in Rice_Production (TotalNPK -10%),Jowar_Yield (TotalNPK +10%),Jowar_Yield (TotalNPK -10%),...,Percent Change in Jowar_Yield (TotalNPK +20%),Percent Change in Jowar_Yield (TotalNPK -20%),Maize_Production (TotalNPK +20%),Maize_Production (TotalNPK -20%),Normalized Maize_Production (TotalNPK +20%),Normalized Maize_Production (TotalNPK -20%),Percent Change in Maize_Production (TotalNPK +20%),Percent Change in Maize_Production (TotalNPK -20%),New Stability(NPK + 20%)_Rice_Jowar_Maize,New Stability(NPK - 20%)_Rice_Jowar_Maize
0,BENGALURU,0.313061,68479.019211,57507.638667,0.122840,0.103039,1928.406967,1603.425316,616.168067,570.414310,...,-58.476612,-64.422519,72303.019039,48389.473335,0.129741,0.086584,2411.393506,1580.773648,0.955632,0.972793
1,BENGALURU(R),0.742141,57992.196208,48927.510755,0.103914,0.087555,2363.559737,1978.483889,572.435052,534.632751,...,inf,inf,59835.351903,40077.695244,0.107240,0.071584,40.514646,-5.883345,0.973031,0.984393
2,RAMANAGARA,0.600687,27479.191334,23962.324950,0.048847,0.042500,126.576446,97.578537,445.187197,430.520870,...,inf,inf,23558.780060,15893.314016,0.041772,0.027938,125.442871,52.089129,0.819324,0.881922
3,CHITRADURGA,0.419056,101029.330849,84139.711825,0.181583,0.151103,2112.644127,1742.744455,751.912067,681.477582,...,-20.007184,-34.323136,111001.722216,74188.608786,0.199581,0.133144,-59.186792,-72.722269,0.666271,0.779886
4,DAVANAGERE,0.106020,266084.831886,219185.121765,0.479460,0.394820,-45.642060,-55.223109,1440.240158,1244.655111,...,-29.024795,-47.076023,307234.370105,205010.374046,0.553723,0.369239,-48.237305,-65.459953,0.787247,0.860537


In [ ]:
# List of districts in the dataset
districts = df['District'].tolist()

# List of columns to visualize
columns_to_visualize = ['Rice_Production', 'Jowar_Yield', 'Maize_Production', 'SowingSeedsDistributed_Jowar', 'SowingSeedsDistributed_Maize']

colors = Category20[len(columns_to_visualize)]

# Create a figure for each column
for column, color in zip(columns_to_visualize, colors):
    # Create a new plot
    plot = figure(title=f'{column} vs District', x_range=districts, sizing_mode='stretch_width', toolbar_location=None, tools='')

    # Plot the data
    source = ColumnDataSource(data=dict(x=districts, y=df[column]))
    plot.vbar(x='x', top='y', width=0.8, color=color, legend_label=column, source=source)

    # Add hover tooltips
    hover = HoverTool()
    hover.tooltips = [('District', '@x'), (column, '@y')]
    plot.add_tools(hover)

    # Styling
    plot.xaxis.major_label_orientation = "vertical"
    plot.yaxis.axis_label = column
    plot.legend.location = 'top_right'
    plot.legend.click_policy = 'hide'

    # Show the plot
    show(plot)


In [ ]:
from bokeh.models import FactorRange
from bokeh.transform import factor_cmap

# Assuming df is your DataFrame

# List of crops
crops = ['Rice_Production', 'Jowar_Yield', 'Maize_Production']

# Create a new plot
plot = figure(title='Production Comparison by Crop and District', x_range=FactorRange(*districts),toolbar_location=None, tools='')

# Plot the data
source = ColumnDataSource(df)
plot.vbar_stack(crops, x='District', width=0.9, color=Category20[len(crops)], source=source, legend_label=crops)

# Add hover tooltips
hover = HoverTool()
hover.tooltips = [('District', '@District')] + [(crop, f'@{crop}') for crop in crops]
plot.add_tools(hover)

# Styling
plot.xaxis.major_label_orientation = "vertical"
plot.yaxis.axis_label = "Production"
plot.legend.location = 'top_left'
plot.legend.click_policy = 'hide'

# Show the plot
show(plot)
